In [1]:
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [15]:
seoul_sch = pd.read_csv('/Users/kimjisu/Desktop/seoulbigcam/Preprocessed Data/서울시 초:중:고 학교명.csv')
seoul_sch = seoul_sch[['학교종류명', '학교명', '도로명주소']]
seoul_sch = seoul_sch.drop_duplicates()
seoul_sch = seoul_sch.reset_index(drop=True)
seoul_sch

,학교종류명,학교명,도로명주소
0,초등학교,서울자곡초등학교,서울특별시 강남구 자곡로 150
1,초등학교,서울율현초등학교,서울특별시 강남구 밤고개로21길 15
2,중학교,세곡중학교,서울특별시 강남구 자곡로 81
3,초등학교,서울세명초등학교,서울특별시 강남구 자곡로 55
4,중학교,휘문중학교,서울특별시 강남구 역삼로 541
...,...,...,...
1403,고등학교,송곡여자고등학교,서울특별시 중랑구 양원역로 67
1404,고등학교,송곡고등학교,서울특별시 중랑구 양원역로 50
1405,고등학교,태릉고등학교,서울특별시 중랑구 신내로23길 36
1406,고등학교,중화고등학교,서울특별시 중랑구 봉화산로27길 62


In [16]:
# 주소로 위도, 경도 좌표 추출하기

addrs = seoul_sch['도로명주소']

# 가입 없이 주소->좌표 변환
from geopy.geocoders import Nominatim

list = []

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

In [29]:
for i in range(1408):
    try:
        crd = geocoding(addrs[i])
        if crd is not None:
            list.append(crd)
        else:
            list.append(None)
            print('{} 번째 좌표를 불러오지 못함'.format(i))
    
        if i % 200 == 0:
            print('{} 번째 좌표 정보 불러오는 중...'.format(i))

    except Exception as e:
        list.append(None)
        print('{} 번째 데이터 오류 발생'.format(i))

add = pd.DataFrame(list)
seoul_sch['x'] = add['lng']
seoul_sch['y'] = add['lat']

0 번째 좌표 정보 불러오는 중...
200 번째 좌표 정보 불러오는 중...


KeyboardInterrupt: 

In [23]:
seoul_sch[seoul_sch['x'].isna()]

,학교종류명,학교명,도로명주소,x,y
1397,초등학교,금성초등학교,서울특별시 중랑구 신내로21길 55,NaN,NaN
1398,고등학교,송곡관광고등학교,서울특별시 중랑구 양원역로 67,NaN,NaN
1399,고등학교,원묵고등학교,서울특별시 중랑구 숙선옹주로 82,NaN,NaN
1400,고등학교,이화여자대학교병설미디어고등학교,서울특별시 중랑구 망우로73길 56,NaN,NaN
1401,고등학교,신현고등학교,서울특별시 중랑구 봉화산로 174,NaN,NaN
1402,고등학교,혜원여자고등학교,서울특별시 중랑구 봉우재로58길 39,NaN,NaN
1403,고등학교,송곡여자고등학교,서울특별시 중랑구 양원역로 67,NaN,NaN
1404,고등학교,송곡고등학교,서울특별시 중랑구 양원역로 50,NaN,NaN
1405,고등학교,태릉고등학교,서울특별시 중랑구 신내로23길 36,NaN,NaN
1406,고등학교,중화고등학교,서울특별시 중랑구 봉화산로27길 62,NaN,NaN


In [15]:
# 도로명 주소로 행정동 받아오기

addrs = seoul_sch['도로명주소']

import requests
import json

locations = []
for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [16]:
city = [] ## 구
town = [] ## 행정동
for i in range(len(locations)):
    
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))
        
    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')
    
    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

In [19]:
city_town = np.array([city,town]).T
df_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
df_temp.head()

,region_2depth_name,region_3depth_h_name
0,강남구,세곡동
1,강남구,세곡동
2,강남구,세곡동
3,강남구,세곡동
4,강남구,대치2동


In [21]:
seoul_sch['gu'] = df_temp['region_2depth_name']
seoul_sch['dong'] = df_temp['region_3depth_h_name']

In [22]:
seoul_sch

,학교종류명,학교명,도로명주소,gu,dong
0,초등학교,서울자곡초등학교,서울특별시 강남구 자곡로 150,강남구,세곡동
1,초등학교,서울율현초등학교,서울특별시 강남구 밤고개로21길 15,강남구,세곡동
2,중학교,세곡중학교,서울특별시 강남구 자곡로 81,강남구,세곡동
3,초등학교,서울세명초등학교,서울특별시 강남구 자곡로 55,강남구,세곡동
4,중학교,휘문중학교,서울특별시 강남구 역삼로 541,강남구,대치2동
...,...,...,...,...,...
3884,고등학교,송곡여자고등학교,서울특별시 중랑구 양원역로 67,NaN,NaN
3888,고등학교,송곡고등학교,서울특별시 중랑구 양원역로 50,NaN,NaN
3892,고등학교,태릉고등학교,서울특별시 중랑구 신내로23길 36,NaN,NaN
3898,고등학교,중화고등학교,서울특별시 중랑구 봉화산로27길 62,NaN,NaN


In [23]:
seoul_sch[seoul_sch['gu'].isna()]

,학교종류명,학교명,도로명주소,gu,dong
1418,평생학교(중)-2년6학기,학력인정 청암중학교(2년제),서울특별시 노원구 중계로 244,NaN,NaN
1419,평생학교(고)-2년6학기,학력인정 청암고등학교(2년제),서울특별시 노원구 중계로 244,NaN,NaN
1421,고등학교,영신간호비즈니스고등학교,서울특별시 노원구 중계로16나길 13,NaN,NaN
1446,고등학교,서울아이티고등학교,서울특별시 노원구 섬밭로 299,NaN,NaN
1467,고등학교,염광여자메디텍고등학교,서울특별시 노원구 월계로45가길 9,NaN,NaN
...,...,...,...,...,...
3884,고등학교,송곡여자고등학교,서울특별시 중랑구 양원역로 67,NaN,NaN
3888,고등학교,송곡고등학교,서울특별시 중랑구 양원역로 50,NaN,NaN
3892,고등학교,태릉고등학교,서울특별시 중랑구 신내로23길 36,NaN,NaN
3898,고등학교,중화고등학교,서울특별시 중랑구 봉화산로27길 62,NaN,NaN


In [11]:
# 가입 없이 주소->좌표 변환
from geopy.geocoders import Nominatim

list = []

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd


crd = geocoding('서울특별시 강남구 자곡로 150')
crd

{'lat': '37.4652137', 'lng': '127.0952705'}

In [14]:
addrs[1]

'서울특별시 강남구 밤고개로21길 15'